# Current Issue

-. My computer is not strong enough to train the model
-. The 1st epoch is still 0% even after waiting for 5 hours of execution

# Possible solutions:

-. Use additional GPUs
-. Execute the code in cloud services like AWS / GCP / Azure
-. Modify the model parameters (most recommended)

# Import Modules for the Deep Learning Model

In [1]:
# install
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install tensorflow
!pip install torch
!pip install keras
!pip install sklearn
!pip install tqdm
!pip install io


# BERT imports
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io


# Feature Engineering + Data Preparation

In [2]:
# Adjust your path variables according to your files location

path = r"C:\Users\Kevin Christian\Desktop\NUS\NUS\Modules\3rd Semester\CS3244\Project\Datasets\\"

In [3]:
train = pd.read_csv(path + "train_text_emoji_clean.csv", sep = '\t')
test = pd.read_csv(path + "test_text_emoji_clean.csv", sep = '\t')
valid = pd.read_csv(path + "valid_text_emoji_clean.csv", sep = '\t')

train["tweets"] = train["annotations,tweets,labels"]
test["tweets"] = test["annotations,tweets,labels"]
valid["tweets"] = valid["annotations,tweets,labels"]

for i in range(len(train)):
    info = train["annotations,tweets,labels"][i].rsplit(",", 1)
    labels = info[1]
    semi = info[0]
    final = semi.split(",", 1)
    message = final[1]
    train["tweets"][i] = message

for i in range(len(test)):
    info = test["annotations,tweets,labels"][i].rsplit(",", 1)
    labels = info[1]
    semi = info[0]
    final = semi.split(",", 1)
    message = final[1]
    test["tweets"][i] = message
    
for i in range(len(valid)):
    info = valid["annotations,tweets,labels"][i].rsplit(",", 1)
    labels = info[1]
    semi = info[0]
    final = semi.split(",", 1)
    message = final[1]
    valid["tweets"][i] = message
    

    
train["annotations"] = train["annotations,tweets,labels"].astype(str).str[0]
test["annotations"] = test["annotations,tweets,labels"].astype(str).str[0]
valid["annotations"] = valid["annotations,tweets,labels"].astype(str).str[0]

# Assign the messages as the main feature for prediction

X_train = train["tweets"]
X_test = test["tweets"]
X_valid = valid["tweets"]

# Extract the first emoji only if there are multiple emojis

#train['result'] = train['annotations'].astype(str).str[0]
#test['result'] = test['annotations'].astype(str).str[0]
#valid['result'] = valid['annotations'].astype(str).str[0]

# Encode the classifications into integers

train['classification'] = pd.factorize(train["annotations"])[0]
test['classification'] = pd.factorize(test["annotations"])[0]
valid['classification'] = pd.factorize(valid["annotations"])[0]

y_train = train['classification']
y_test = test['classification']
y_valid = valid['classification']

In [5]:
emojis = train["annotations"].unique()

# This emojis can be used to convert the predicted number into its respective emoji

emojis

array(['😂', '😭', '😍', '🙄', '❤', '😊', '😩', '🤔', '😘', '🏽', '💯', '🙃'],
      dtype=object)

In [6]:
print(X_train)

print("==================================================")

print(y_train)

0                               school is so dead o my god
1        lol I've been told mine is worse than yours bu...
2        I'm excited to hear them..... That shit is goi...
3        Damn alycia knows everything even indirect tweets
4                   x_juicebox: That sound like everything
                               ...                        
59900    FinesseYoAss: Or That Picture Was Just Cute To...
59901                   I'm otw wit some gas still hate me
59902                                  Yeahhh you kinda do
59903                       nothing wit youuuuu raymundooo
59904    dont be that person who parks right next to me...
Name: tweets, Length: 59905, dtype: object
0         0
1         0
2         0
3         0
4         0
         ..
59900    11
59901    11
59902    11
59903    11
59904    11
Name: classification, Length: 59905, dtype: int64


In [7]:
print(X_test)

print("==================================================")

print(y_test)

0          The conversation is so deep i saw adele rolling
1        "Saras_OREO: Lord, we're coo asf lol Just wait...
2                                                  I can't
3        These little kids talking about going to Canad...
4                                       only would do that
                               ...                        
15179                           The ball hates Croatia smh
15180                        Sexy bitch I hope she bout it
15181    I didn't realize that my tweets were private a...
15182                         lowkey annoyed for no reason
15183                          appreciate it while it last
Name: tweets, Length: 15184, dtype: object
0         0
1         0
2         0
3         0
4         0
         ..
15179    11
15180    11
15181    11
15182    11
15183    11
Name: classification, Length: 15184, dtype: int64


In [8]:
print(X_valid)

print("==================================================")

print(y_valid)

0                             hahahah you guys crack me up
1                                                 I'm dead
2                    ate girl! The tweet was before TMC pa
3        Also who would've thought that Will Griggs on ...
4                           I'm not fooling w. you tonight
                               ...                        
15328    Hopefully I don't fall asleep in the toilets i...
15329    Excited for the lake. Not excited to wake up a...
15330    mom just told me shes leaving early to go to a...
15331          all these double shifts are killing my vibe
15332    KamryDeLeon: I love when my legs are shaved bu...
Name: tweets, Length: 15333, dtype: object
0         0
1         0
2         0
3         0
4         0
         ..
15328    11
15329    11
15330    11
15331    11
15332    11
Name: classification, Length: 15333, dtype: int64


In [9]:
# In BERT, always remember to include [CLS] and [SEP] for every text.

sentences_train = ["[CLS] " + query + " [SEP]" for query in X_train]
sentences_test = ["[CLS] " + query + " [SEP]" for query in X_test]
sentences_valid = ["[CLS] " + query + " [SEP]" for query in X_valid]
print(sentences_train[1])
print(sentences_test[1])
print(sentences_valid[1])

[CLS] lol I've been told mine is worse than yours but I don't believe that [SEP]
[CLS] "Saras_OREO: Lord, we're coo asf lol Just waiting for break so we can chill" [SEP]
[CLS] I'm dead [SEP]


In [10]:
#Tokenize all the sentences in the train, test, valid

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts_train = [tokenizer.tokenize(sent) for sent in sentences_train]
tokenized_texts_test = [tokenizer.tokenize(sent) for sent in sentences_test]
tokenized_texts_valid = [tokenizer.tokenize(sent) for sent in sentences_valid]
print ("Tokenize the first sentence of the training:")
print (tokenized_texts_train[0])

Tokenize the first sentence of the training:
['[CLS]', 'school', 'is', 'so', 'dead', 'o', 'my', 'god', '[SEP]']


In [11]:
# Set the maximum sequence length. 
MAX_LEN = 128
# Pad our input tokens
input_ids_train = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_train],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids_valid = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_valid],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids_train = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_train]
input_ids_train = pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids_test = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test]
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids_valid = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_valid]
input_ids_valid = pad_sequences(input_ids_valid, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [12]:
# Create attention masks
attention_masks_train = []
attention_masks_test = []
attention_masks_valid = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids_train:
  seq_mask = [float(i>0) for i in seq]
  attention_masks_train.append(seq_mask)

for seq in input_ids_test:
  seq_mask = [float(i>0) for i in seq]
  attention_masks_test.append(seq_mask)

for seq in input_ids_valid:
  seq_mask = [float(i>0) for i in seq]
  attention_masks_valid.append(seq_mask)

In [13]:
train_inputs = input_ids_train
test_inputs = input_ids_test
validation_inputs = input_ids_valid

train_labels = y_train
test_labels = y_test
validation_labels = y_valid

train_masks = attention_masks_train
test_masks = attention_masks_test
validation_masks = attention_masks_valid

In [14]:
# Convert all of our data into torch tensors, the required datatype for our model
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

# Select a batch size for training. 
batch_size = 32

# Create an iterator of our data with torch DataLoader 
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


# Model Training

In [17]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=12)

model.parameters

#model.cuda()

<bound method Module.parameters of BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout)

In [ ]:
# BERT fine-tuning parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
  
# Store our loss and accuracy for plotting
train_loss_set = []
# Number of training epochs 
epochs = 4

# BERT training loop
for _ in trange(epochs, desc="Epoch"):  
  
  ## TRAINING
  
  # Set our model to training mode
  model.train()  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    #batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1
  print("Train loss: {}".format(tr_loss/nb_tr_steps))
       
  ## VALIDATION

  # Put model in evaluation mode
  model.eval()
  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0
  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    #batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

# plot training performance
plt.figure(figsize=(15,8))
plt.title("Training loss")
plt.xlabel("Batch")
plt.ylabel("Loss")
plt.plot(train_loss_set)
plt.show()

# Model Testing

In [ ]:
batch_size = 32  
prediction_data = TensorDataset(test_inputs, test_masks, test_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

## Prediction on test set
# Put model in evaluation mode
model.eval()
# Tracking variables 
predictions , true_labels = [], []
# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)
  
# Import and evaluate each test batch using Matthew's correlation coefficient
from sklearn.metrics import matthews_corrcoef
matthews_set = []
for i in range(len(true_labels)):
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  matthews_set.append(matthews)
  
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

print('Classification accuracy using BERT Fine Tuning: {0:0.2%}'.format(matthews_corrcoef(flat_true_labels, flat_predictions)))

# Save The Best Model

In [ ]:
model.save_pretrained('./best_model')